In [239]:
import pandas as pd
import numpy as np

In [240]:
#extract data from the San Diego AirBnb dataset found here: https://www.kaggle.com/datasets/thedevastator/san-diego-airbnb-listings-august-2019?resource=download
data = pd.read_csv('raw_data/listings.csv') 
print(data.head(10))

   index        id                            listing_url  \
0      0  33159143  https://www.airbnb.com/rooms/33159143   
1      1  17138468  https://www.airbnb.com/rooms/17138468   
2      2  21898446  https://www.airbnb.com/rooms/21898446   
3      3  25948680  https://www.airbnb.com/rooms/25948680   
4      4   1756516   https://www.airbnb.com/rooms/1756516   
5      5  33395869  https://www.airbnb.com/rooms/33395869   
6      6  14139907  https://www.airbnb.com/rooms/14139907   
7      7  28055565  https://www.airbnb.com/rooms/28055565   
8      8  19138413  https://www.airbnb.com/rooms/19138413   
9      9  30342963  https://www.airbnb.com/rooms/30342963   

                                                name  \
0   PRIVATE ROOM, SAFE NEIGHBORHOOD IN SCRIPPS RANCH   
1                                      NOT AVAILABLE   
2                          Townhome in Pacific Beach   
3     Spacious furnished 1 BR with tons of amenaties   
4                          A Spacious luxury ret

/var/folders/bx/0m6t3kp16n10mbf470qqnmjw0000gn/T/ipykernel_2049/1576862903.py:2: DtypeWarning: Columns (1,13,14,25,44,55,57) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('raw_data/listings.csv')


In [178]:
columns = data.columns
print(columns)

Index(['index', 'id', 'listing_url', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'thumbnail_url', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'street', 'neighbourhood',
       'neighbourhood_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'nightly_price', 'price_per_stay', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum

In [241]:
cols_needed = ['index', 'id','name', 'description','host_id','host_name',
               'host_listings_count','city','neighbourhood','zipcode','property_type',
                'room_type', 'accommodates','bathrooms', 'bedrooms', 'amenities', 
                'square_feet', 'nightly_price', 'price_per_stay', 'number_of_reviews',
                'review_scores_rating'
                ] 
data = data.drop(columns = [col for col in columns if col not in cols_needed])

In [346]:
properties_df = data[['id', 'host_id','name', 'property_type','nightly_price', 'zipcode',
                     'city','neighbourhood','square_feet','accommodates',
                     'bathrooms', 'bedrooms', 'amenities']]
properties_df = properties_df.rename(columns={"id": "property_id", "name": "property_name",
                               'nightly_price':'price_per_night',
                                })

#host_no_of_properties likely to be inaccurate
owner_df = data[['host_id','host_name']]
owner_df = owner_df.rename(columns = {'host_name':'name'})


## Create owner table

In [347]:
owner_df = owner_df.drop_duplicates()
print(owner_df['host_id'].nunique())
print(owner_df['host_id'].isna().sum())

8011
0


In [348]:
#due to mockaroo's maximimum output of 1000, will limit number of owners to 500.
#property entries with this host_id will also be removed accordingly
owner_df = owner_df.sample(n=500, random_state=1)

In [349]:
owner_df = owner_df.reset_index(drop=True)
owner_df = owner_df.reset_index()

In [350]:
raw_owner_df = pd.read_csv('/Users/leeey/Downloads/prototype-it2002/raw_data/owner_table_raw.csv')
raw_owner_df.head(2)

,email,password,credit_card_no
0,nmacfie0@yelp.com,vNsynoMv,5.048377e+15
1,zeckly1@youtu.be,nsHJPkaDDrUH,5.048378e+15


In [351]:
raw_owner_df = raw_owner_df.reset_index()

In [352]:
owner_table_df = pd.merge(owner_df,raw_owner_df, on='index')
print(owner_table_df.head(2))

   index   host_id   name              email      password  credit_card_no
0      0  27699479  Corin  nmacfie0@yelp.com      vNsynoMv    5.048377e+15
1      1  15614004    Eva   zeckly1@youtu.be  nsHJPkaDDrUH    5.048378e+15


In [353]:
owner_table_df.shape

(500, 6)

In [354]:
#select 100 owners who also have a guest account to add to guests_table
owners_who_are_guests = owner_table_df.sample(n=100, random_state=1)
owners_who_are_guests = owners_who_are_guests.drop(['index','host_id'],axis=1)

In [355]:
owner_table_df

,index,host_id,name,email,password,credit_card_no
0,0,27699479,Corin,nmacfie0@yelp.com,vNsynoMv,5.048377e+15
1,1,15614004,Eva,zeckly1@youtu.be,nsHJPkaDDrUH,5.048378e+15
2,2,249722267,Sherri,dkeeri2@1688.com,5yAHaiZ,5.048379e+15
3,3,1389938,Scott,catter3@blogs.com,YxP1DLN,5.108755e+15
4,4,21983712,Kylie,mferie4@nasa.gov,76BYVa2,5.108751e+15
...,...,...,...,...,...,...
495,495,259477046,Roger,gstrutleydr@cornell.edu,xmwqBgFY,5.108758e+15
496,496,26508552,Ray,cisseleeds@ask.com,9zUNXhgx6IyV,5.108755e+15
497,497,92866068,Lata,ltregiendt@intel.com,pPTXaPl,5.108751e+15
498,498,9075328,Brianne,cattwaterdu@yelp.com,uWo9Ty,5.048379e+15


In [356]:
#remove unnecessary columns for owners
owner_table_df = owner_table_df.drop(['index','credit_card_no'], axis=1)
owner_table_csv_df = owner_table_df.drop(['host_id'], axis=1)

In [357]:
#add quotes for later sql format
owner_table_csv_df['name'] = "'" + owner_table_csv_df['name'].astype(str) + "'"
owner_table_csv_df['email'] = "'" + owner_table_csv_df['email'].astype(str) + "'"
owner_table_csv_df['password'] = "'" + owner_table_csv_df['password'].astype(str) + "'"

In [358]:
owner_table_csv_df

,name,email,password
0,'Corin','nmacfie0@yelp.com','vNsynoMv'
1,'Eva','zeckly1@youtu.be','nsHJPkaDDrUH'
2,'Sherri','dkeeri2@1688.com','5yAHaiZ'
3,'Scott','catter3@blogs.com','YxP1DLN'
4,'Kylie','mferie4@nasa.gov','76BYVa2'
...,...,...,...
495,'Roger','gstrutleydr@cornell.edu','xmwqBgFY'
496,'Ray','cisseleeds@ask.com','9zUNXhgx6IyV'
497,'Lata','ltregiendt@intel.com','pPTXaPl'
498,'Brianne','cattwaterdu@yelp.com','uWo9Ty'


In [359]:
# download csv of owner_table
## owner_table_csv_df.to_csv('owners_table.csv', index=False)

Create guests table

In [360]:
#create guests table from mockaroo data
#500 rows generated. credit_card_no all from the US, 20% of credit_card_no is left blank, 10% of name left blank to be filled in with repeat values
guest_df = pd.read_csv('/Users/leeey/Downloads/prototype-it2002/raw_data/guest_table_raw.csv') 
print(guest_df.head(3))

             name                  email      password  credit_card_no
0    Nanny Dudman       ndudman0@mail.ru       Fdoipja    5.048374e+15
1    Dorey Finkle   dfinkle1@oaic.gov.au   Aim4tREQont    5.048377e+15
2  Erroll Krysiak  ekrysiak2@nbcnews.com  IFRxqF0B0p6J    5.108754e+15


In [361]:
guest_df.shape

(500, 4)

In [362]:
#ensure that emails are all unique
guest_df = guest_df.drop_duplicates(subset=['email'], keep='first')
guest_df.shape

(500, 4)

In [363]:
#fill in repeat values of name using closest row above each NA value
print(guest_df['name'].isna().sum())
guest_df['name'] = guest_df['name'].fillna(method='ffill')
print(guest_df['name'].isna().sum())

51
0


In [364]:
guest_df = guest_df.append(owners_who_are_guests, ignore_index = True)

/var/folders/bx/0m6t3kp16n10mbf470qqnmjw0000gn/T/ipykernel_2049/447354538.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  guest_df = guest_df.append(owners_who_are_guests, ignore_index = True)


In [365]:
# check that all emails are still unique
print(guest_df['email'].value_counts().sum())

# check for percentage of blanks in credit_card_no
print(guest_df['credit_card_no'].isna().sum())

600
121


In [366]:
#add quotes for later sql format
guest_df['name'] = "'" + guest_df['name'].astype(str) + "'"
guest_df['email'] = "'" + guest_df['email'].astype(str) + "'"
guest_df['password'] = "'" + guest_df['password'].astype(str) + "'"

In [367]:
guest_df['credit_card_no'] = guest_df['credit_card_no'].fillna('NULL')

In [368]:
# export guest_df to csv format
## guest_df.to_csv('guests_table.csv', index=False)

## Create properties table

In [369]:
print(properties_df.shape)
properties_df = properties_df[properties_df.host_id.isin(owner_table_df.host_id)]
print(properties_df.shape)
print(properties_df.columns)

(13052, 13)
(819, 13)
Index(['property_id', 'host_id', 'property_name', 'property_type',
       'price_per_night', 'zipcode', 'city', 'neighbourhood', 'square_feet',
       'accommodates', 'bathrooms', 'bedrooms', 'amenities'],
      dtype='object')


In [370]:
# need to change host_id to host email instead
properties_df = properties_df.merge(owner_table_df[['host_id','email']], how='inner', on= 'host_id') 
properties_df = properties_df.loc[:, ['property_id', 'email', 'property_name', 'property_type',
       'price_per_night', 'zipcode', 'city', 'neighbourhood', 'square_feet',
       'accommodates', 'bathrooms', 'bedrooms', 'amenities']]

In [371]:
properties_df.columns

Index(['property_id', 'email', 'property_name', 'property_type',
       'price_per_night', 'zipcode', 'city', 'neighbourhood', 'square_feet',
       'accommodates', 'bathrooms', 'bedrooms', 'amenities'],
      dtype='object')

In [374]:
#add quotes for later sql format
properties_str_cols = ['email', 'property_name', 'property_type','city', 'neighbourhood','amenities']
for col in properties_str_cols:
    properties_df[col] = "'" + properties_df[col].astype(str) + "'"

In [372]:
properties_num_cols = ['property_id','price_per_night', 'zipcode','square_feet',
                       'accommodates', 'bathrooms', 'bedrooms']
for col in properties_num_cols:
    properties_df[col] = properties_df[col].fillna('NULL')

In [377]:
properties_df['price_per_night'][0].dtype

dtype('O')

In [376]:
# export properties_df to csv format
properties_df.to_csv('properties_table.csv', index=False)

## Create bookings table

In [225]:
# mockaroo used to generate 100 rows of start_date, duaration and calculated end_date based off first two columns 
# start_date can range from 01/01/2021 to 03/20/2023
# duration can range from 1 to 30 days
booking_df = pd.read_csv('/Users/leeey/Downloads/prototype-it2002/raw_data/bookings_table_raw.csv') 

In [226]:
# generate random combinations of owner email, guest email and property_id
# make sure that properties are not double booked (or maybe dont care then SQL can check for it)
owner_property_df = properties_df[['email', 'property_id']].rename({'email':'owner_email'}, axis =1)
booking_entities_df = guest_df[['email']].rename({'email':'guest_email'}).join(owner_property_df, how = 'cross')
print(booking_entities_df)

                      email                  owner_email property_id
0          ndudman0@mail.ru  tjaumet2k@howstuffworks.com     6372898
1          ndudman0@mail.ru  tjaumet2k@howstuffworks.com     2965336
2          ndudman0@mail.ru        kmcmeekan72@bbc.co.uk     7871918
3          ndudman0@mail.ru            tlampsl@topsy.com    32286661
4          ndudman0@mail.ru           drabat9u@apple.com    29452722
...                     ...                          ...         ...
491395  gullyatt32@army.mil         agraffin4q@opera.com    19273736
491396  gullyatt32@army.mil          ckerrey92@mysql.com     1487384
491397  gullyatt32@army.mil      rgarritley5k@nature.com    34548630
491398  gullyatt32@army.mil         scrasforda9@army.mil    20437139
491399  gullyatt32@army.mil      rsalvadore5o@flavors.me    34649201

[491400 rows x 3 columns]


In [227]:
# select 100 rows randomly to generate 100 bookings randomly
booking_entities_df = booking_entities_df.sample(n=100, random_state=1)
booking_entities_df = booking_entities_df.reset_index(drop =True)
booking_entities_df = booking_entities_df.reset_index()
print(booking_entities_df.head(2))

   index                    email              owner_email property_id
0      0     hmcvicard@zimbio.com  moakly38@friendfeed.com    26630989
1      1  oarckolldv@symantec.com      alevett65@wufoo.com    28781817


In [228]:
booking_df = booking_df.reset_index()

In [229]:
booking_table_df = pd.merge(booking_entities_df,booking_df, on='index')

In [289]:
booking_table_df = booking_table_df.drop(['index'], axis=1)

In [290]:
booking_table_df

,email,owner_email,property_id,start_date,duration,end_date
0,hmcvicard@zimbio.com,moakly38@friendfeed.com,26630989,2021-08-30 19:46:16,10,2021-09-09 19:46:16
1,oarckolldv@symantec.com,alevett65@wufoo.com,28781817,2021-05-17 22:26:15,24,2021-06-10 22:26:15
2,tblake3x@amazon.co.jp,bkaretas@irs.gov,35492459,2021-12-13 14:18:19,20,2022-01-02 14:18:19
3,ccatterson30@360.cn,bpentin9q@google.es,11720963,2021-11-06 11:25:42,9,2021-11-15 11:25:42
4,acartmail8j@omniture.com,aizac7i@yandex.ru,26196827,2022-07-12 18:56:42,3,2022-07-15 18:56:42
...,...,...,...,...,...,...
95,jresdalea3@sciencedirect.com,dlidgeycd@youtube.com,37569194,2021-02-25 08:38:40,1,2021-02-26 08:38:40
96,vbartak3z@shareasale.com,vpeytoco@princeton.edu,960655,2021-12-26 11:17:42,27,2022-01-22 11:17:42
97,mhackett2f@vinaora.com,moakly38@friendfeed.com,19487368,2023-01-01 13:23:27,19,2023-01-20 13:23:27
98,ssebastian3r@google.pl,wrimbault2s@ucla.edu,2483060,2022-07-09 23:46:24,16,2022-07-25 23:46:24


In [291]:
# export booking_table_df to csv format
booking_table_df.to_csv('bookings_table.csv', index=False)

## Create wishlist table

In [232]:
wishlist_df = guest_df[['email']].rename({'email':'guest_email'}, axis=1).join(properties_df[['property_id']], how='cross')
print(wishlist_df)

                guest_email property_id
0          ndudman0@mail.ru     6372898
1          ndudman0@mail.ru     2965336
2          ndudman0@mail.ru     7871918
3          ndudman0@mail.ru    32286661
4          ndudman0@mail.ru    29452722
...                     ...         ...
491395  gullyatt32@army.mil    19273736
491396  gullyatt32@army.mil     1487384
491397  gullyatt32@army.mil    34548630
491398  gullyatt32@army.mil    20437139
491399  gullyatt32@army.mil    34649201

[491400 rows x 2 columns]


In [233]:
#create 500 wishlist entries by sampling randomly from all possible combinations of guest_email and property_id
wishlist_df =  wishlist_df.sample(n=500, random_state=2)

In [238]:
# export wishlist_df to csv format
##wishlist_df.to_csv('wishlist_table.csv', index=False)